In [15]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, CONNECTIONS_CONFIG, QWEN_1_5_B

cfg = CONNECTIONS_CONFIG
cfg.train_batch_size = 2
cfg.num_generations = 2
cfg.model_id_or_path = QWEN_1_5_B
cfg.notebook_mode = True
cfg.max_samples = 1000

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
wrapper = TrainerWrapper(cfg)
wrapper.init_model()

2025-04-12 09:40:17.456 | INFO     | trl_wrapper.trainer_wrapper:__init__:285 - Setting padding side to: left
2025-04-12 09:40:17.456 | INFO     | trl_wrapper.trainer_wrapper:init_model:296 - Using device: mps
2025-04-12 09:40:17.457 | INFO     | trl_wrapper.trainer_wrapper:init_model:310 - Loading model Qwen/Qwen2.5-1.5B-Instruct with attn_impl: sdpa


In [17]:
from model.reasoning import ConnectionsDataModule

In [18]:
conn = ConnectionsDataModule(wrapper.tokenizer, wrapper.config)

2025-04-12 09:40:22.095 | INFO     | trl_wrapper.wrapper_config:__init__:145 - Cache dir: ../dataset_caches/connections_data_module


In [19]:
wrapper.init_data_module()

2025-04-12 09:40:22.148 | INFO     | trl_wrapper.wrapper_config:__init__:145 - Cache dir: ../dataset_caches/connections_data_module
Map: 100%|██████████| 100000/100000 [00:07<00:00, 12690.86 examples/s]


In [20]:
wrapper.init_trainer()

2025-04-12 09:41:16.580 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:373 - Saving output to: ./runs/04-12-9-41-741094-qwen2.5-1.5b-instruct-
2025-04-12 09:41:16.582 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:396 - Initializing trainer, run_name: 04-12-9-41-741094-qwen2.5-1.5b-instruct-, wandb project: qwen-connections-grpo
2025-04-12 09:41:16.582 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:479 - Using vllm: False


In [21]:
wrapper.train()

2025-04-12 09:41:16.688 | INFO     | trl_wrapper.trainer_wrapper:train:644 - Starting training.
/Users/brianfitzgerald/Documents/GitHub/smolmodels/.venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
2025-04-12 09:42:01.576 | INFO     | model.reasoning:xmlcount_reward_func:59 - XML count rewards: [-0.09699999999999998, 0.11299999999999999]
2025-04-12 09:42:01.576 | INFO     | model.reasoning:strict_format_reward_func:105 - Strict format rewards: [0.25, 0.25]
2025-04-12 09:42:01.577 | INFO     | model.reasoning:soft_group_reward:203 - Soft accuracy scores: [0.375, 0.5]
2025-04-12 09:42:01.577 | INFO     | model.reasoning:group_size_reward:249 - Group size rewards: [0.25, 1.0]
2025-04-12 09:42:01.578 | INFO     | model.reasoning:hard_group_reward:215 - Hard accuracy scores: [0.25, 0.0]
2025-04-12 09:42:01.578 | INFO     | model.reasoning:n_groups_reward:260 - Number of groups rewa

ZeroDivisionError: float division by zero